### Create permutations of possible parameters

In [ ]:
from itertools import product
import numpy as np

input_nb_path = './minibatch-kmeans.ipynb'

parameter_dict = {
'surface_dimensions' : [[9,9], [13,13]],
'time_constant' : [10000, 25000, 50000],
'dropout_probability' : [0, 0.5, 0.9],
'n_of_centers' : [36, 49, 64],
}

value_permutations = list(product(*list(parameter_dict.values())))
print('First set of ' + str(len(value_permutations)) + ' parameter combinations: '+ str(dict(zip(parameter_dict.keys(), value_permutations[0]))))

short_keys = ['_'+k[:4]+'_' for k in list(parameter_dict.keys())] # will be used for nb filenames

### Execute notebooks in parallel

In [ ]:
import multiprocessing
import os, glob
import time
from functools import partial
import papermill as pm
from joblib import Parallel, delayed, parallel_backend

def process_notebook(params, short_keys, parameter_dict, input_nb_path):
    string = ''
    for index, key in enumerate(short_keys):
        string += key + str(params[index])
    string += '.ipynb'

    param_dict = dict(zip(parameter_dict.keys(), params))
    print(param_dict)
    param_dict['file_name'] = string
    
    output = './milled_nbs/' + string
    pm.execute_notebook(
       input_nb_path,
       output,
       parameters = param_dict
    )

start_time = time.time()
#with parallel_backend("loky", inner_max_num_threads=2):
Parallel(n_jobs=len(value_permutations))(delayed(process_notebook)(params, short_keys, parameter_dict, input_nb_path) for params in value_permutations)
print("--- %s seconds ---" % (time.time() - start_time))
#for filename in glob.glob("./milled_nbs/_surf*"):
#    os.remove(filename) 